In [ ]:
import numpy as np
import time

In [ ]:
goal = 'foo bar foo bar'
SEARCH_SPACE_START, SEARCH_SPACE_END = 32, 128
SEARCH_SPACE = list('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890 ')
init_pop_size = 1000
breed_rate = .8
crossover_probability = .1
mutation_probability = .01
epsilon_sqrt = 0.0001


In [ ]:
def initialize_population(init_pop_size, goal):
    return np.array([''.join([i for i in np.random.choice(SEARCH_SPACE, len(goal))]) for i in range(init_pop_size)])

In [ ]:
def fitness(individual):
#     print(individual)
    return sum([np.absolute(ord(goal[i])-ord(individual[i])) for i in range(len(individual))])

In [ ]:
def crossover(indiv_1, indiv_2):
    crossover_point = np.random.randint(0, len(indiv_1), 1)[0]
    return ''.join(list([str(indiv_1[i]) for i in range(crossover_point)] + [str(indiv_2[i]) for i in range(crossover_point, len(indiv_2))]))

In [ ]:
def mutation(curr_pop, mutation_prob):
    def indiv_mutate(individual, mutation_prob):
        l = list(individual)
        for i in range(len(l)):
            if np.random.uniform(0,1) < mutation_prob:
                l[i] = np.random.choice(SEARCH_SPACE, len(goal))[0]
        return ''.join(l)
    return np.array([indiv_mutate(i, mutation_prob) for i in curr_pop])

In [ ]:
def isAmongGeneration(goal, gen):
    if goal in gen:
        return True
    else:
        return False

In [ ]:
def getGeneIndex(goal, gen):
    count = 0
    for gene in gen:
        if gene == goal:
            return count
        else:
            count += 1
    return -1

In [ ]:
def genetic_generation(goal, init_pop_size, mutation_probability, breed_rate):
    curr_pop = initialize_population(init_pop_size, goal)
    curr_fitness = np.array(list(map(fitness, curr_pop)))
    curr_fitness_distr = (1/curr_fitness)/sum(1/curr_fitness)
    generation = 0
    fittest_individual = curr_pop[min(curr_fitness)]
    stored_pop = []
    while(fittest_individual is not goal and curr_pop != 0 and not isAmongGeneration(goal, stored_pop)):
        generation += 1
        print("Generation " + str(generation) + ":", str(fittest_individual) + " Fitness: " + str(max(curr_fitness)))
        new_pop = []
        fitness_enum = sorted([(curr_pop[i],curr_fitness[i]) for i in range(len(curr_pop))], key=lambda x: x[1])[::-1][:int(.8)*len(curr_pop)]
        new_pop = new_pop + fitness_enum
        for i in range(0, int(len(curr_pop)-len(fitness_enum))):
            parent_1, parent_2 = tuple([curr_pop[np.random.choice(len(curr_pop), 1, p=curr_fitness_distr)], curr_pop[np.random.choice(len(curr_pop), 1, p=curr_fitness_distr)]])
            new_pop.append(''.join([i for i in parent_1]))
        new_pop = mutation(new_pop, mutation_probability)
        stored_pop = new_pop
        curr_pop = np.array(new_pop)
        curr_fitness = np.array(list(map(fitness, curr_pop)))
        curr_fitness_distr = (1/(curr_fitness+epsilon_sqrt))/sum(1/(curr_fitness+epsilon_sqrt))
        fittest_individual = curr_pop[np.argmax(curr_fitness)]
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print("Found goal in generation " + str(generation) + " index number " + str(getGeneIndex(goal, stored_pop)) + ": " + '"' + goal + '"')

In [ ]:
genetic_generation(goal, init_pop_size, mutation_probability, breed_rate)